Ссылка на репозиторий проекта:
https://github.com/andrew-tvorogov/practicum_masterskaya_1

## Рекомендации для инвесторов о целесообразности инвестирования в стартапы на основе данных о стартапах, функционировавших в период с 1970 по 2018 годы.

<b>Заказчик исследования:</b>
- yandex practicum

<b>Источник данных:</b>
- псевдо-реальные (реальные данные в которые добавлена синтетическая составляющая) данные о стартапах, функционировавших в период с 1970 по 2018 годы

<b>Содержание данных:</b>
- данные о стартапах;

<b>Порядок проведения исследования:</b>
- загрузить данные;
- изучить данные;
- выполнить предобработку данных;
- полноценный разведочный анализ
    - провести исследовательский анализ;
    - провести корреляционный анализ признаков датасета;
- разработка новых синтетических признаков;
- проверка на мультиколлинеарность;
- отбор финального набора обучающих признаков;
- выбрать лучшую модель (выбор и обучение моделей);
    - создать пайплан обработки признаков;
    - создать пайплайн выбора лучшей модели.
- итоговая оценка качества предсказания лучшей модели;
- анализ важности ее признаков;
- сделать общий вывод на основе готовых моделей (подготовка отчета по исследованию).

<b>Задача:</b>
<p>Разработать решение, которое позволит предсказать какие стартапы закроются.</p>

<b>Цель исследования:</b>
- снижение рисков при инвестировании в стартап;
- получение прибыли;

Данные доступны в файле ``kaggle_startups_train_28062024.csv``

<b>Описание данных:</b>

- ``name`` название стартапа;
- ``category_list`` cписок категорий, к которым относится стартап;
- ``funding_total_usd`` общая сумма финансирования в USD;
- ``status`` татус стартапа (закрыт или действующий);
- ``country_code`` код страны;
- ``state_code`` код штата;
- ``region`` регион;
- ``city`` город;
- ``funding_rounds`` количество раундов финансирования;
- ``founded_at`` дата основания;
- ``first_funding_at`` дата первого раунда финансирования;
- ``last_funding_at`` дата последнего раунда финансирования;
- ``closed_at`` дата закрытия стартапа (если применимо);
- ``lifetime`` время существования стартапа в днях.ртапа в днях.в днях
</pre>

<b>Дополнительно:</b>

Для участия в соревновании на ресурсе Kaggle доступны дополнительные файлы:
- ``kaggle_startups_train_28062024.csv``
- ``kaggle_startups_sample_submit_28062024.csv``

In [1]:
# установка пакетов
# SHapley Additive exPlanations
!pip install shap --quiet 

# correlation analyzer library
!pip install phik --quiet

In [2]:
# Загрузка библиотек
import pandas as pd
import matplotlib.pyplot as plt # для диаграмм
import math # для диаграмм
import numpy as np # для равномерного распределения значений на диаграмме
import seaborn as sns
import shap # оценка вклада признака
import phik

from sklearn.preprocessing import (
    StandardScaler,
    MinMaxScaler,
    RobustScaler, 
    OneHotEncoder, 
    OrdinalEncoder)

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score

from warnings import simplefilter

In [3]:
# Функции

In [4]:
# инициализация глобальных переменных
RANDOM_STATE = 42 # Deep Thought number
TEST_SIZE = 0.25 # размер выборки

In [5]:
# настройки
pd.options.display.max_columns = None # необходимо чтобы head отобразил все колонки
plt.rcParams['figure.figsize'] = [15, 3] # размеры графика по умолчанию
sns.set(rc={'figure.figsize':(15, 10)}) # размеры графика seaborn по умолчанию
pd.set_option('display.float_format', '{:.2f}'.format) # отобразит в колонках округленные данные до 6-ти знаков

simplefilter(action='ignore', category=FutureWarning) # уберёт предупреждение "Set `keepdims`

## Загрузка и изучение данных

### Загрузка

In [6]:
# загрузка данных в датасеты
try:
    data = pd.read_csv('./datasets/kaggle_startups_train_28062024.csv')
    startups_test = pd.read_csv('./datasets/kaggle_startups_test_28062024.csv')
    startups_submit = pd.read_csv('./datasets/kaggle_startups_sample_submit_28062024.csv')
    #money = pd.read_csv('./datasets/money.csv', sep=';', decimal=",")    
except:
    print("Ошибка. Данные не загружены")

In [7]:
startups_test.head(1)

,name,category_list,funding_total_usd,country_code,state_code,region,city,funding_rounds,first_funding_at,last_funding_at,lifetime
0,Crystalsol,Clean Technology,2819200.00,NIC,17,NaN,NaN,1,2009-07-01,2009-07-01,3501


In [8]:
data.head(50)

,name,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at,closed_at
0,Lunchgate,Online Reservations|Restaurants,828626.00,operating,CHE,25,Zurich,Zürich,2,2009-10-17,2011-05-01,2014-12-01,NaN
1,EarLens,Manufacturing|Medical|Medical Devices,42935019.00,operating,USA,CA,SF Bay Area,Redwood City,4,2005-01-01,2010-05-04,2014-02-25,NaN
2,Reviva Pharmaceuticals,Biotechnology,35456381.00,operating,USA,CA,SF Bay Area,San Jose,3,2006-01-01,2012-08-20,2014-07-02,NaN
3,Sancilio and Company,Health Care,22250000.00,operating,NaN,NaN,NaN,NaN,3,2004-01-01,2011-09-01,2014-07-18,NaN
4,WireTough Cylinders,Manufacturing,NaN,operating,USA,VA,VA - Other,Bristol,1,2010-05-12,2012-02-01,2012-02-01,NaN
5,Connected Sports Ventures,Mobile,4300000.00,operating,USA,NJ,Newark,Princeton,1,2011-04-16,2012-11-12,2012-11-12,NaN
6,Attensity,Analytics|Business Analytics|Social CRM|Social...,90000000.00,operating,USA,CA,SF Bay Area,Redwood City,1,2000-01-01,2014-05-14,2014-05-14,NaN
7,Mesh Networks,Software,4300000.00,operating,USA,TX,Houston,Houston,1,2005-01-01,2014-11-09,2014-11-09,NaN
8,AngioScore,Biotechnology,42000000.00,operating,USA,CA,SF Bay Area,Fremont,2,2003-01-01,2007-10-09,2011-04-20,NaN
9,Vidatronic,Semiconductors,1250500.00,operating,USA,TX,Austin,College Station,2,2010-01-01,2011-08-23,2013-03-21,NaN


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52516 entries, 0 to 52515
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               52515 non-null  object 
 1   category_list      50051 non-null  object 
 2   funding_total_usd  42447 non-null  float64
 3   status             52516 non-null  object 
 4   country_code       47014 non-null  object 
 5   state_code         45753 non-null  object 
 6   region             46157 non-null  object 
 7   city               46157 non-null  object 
 8   funding_rounds     52516 non-null  int64  
 9   founded_at         52516 non-null  object 
 10  first_funding_at   52516 non-null  object 
 11  last_funding_at    52516 non-null  object 
 12  closed_at          4917 non-null   object 
dtypes: float64(1), int64(1), object(11)
memory usage: 5.2+ MB


In [10]:
date_cols = ["founded_at","first_funding_at","last_funding_at","closed_at"]
data[date_cols] = data[date_cols].astype("datetime64[ns]")

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52516 entries, 0 to 52515
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   name               52515 non-null  object        
 1   category_list      50051 non-null  object        
 2   funding_total_usd  42447 non-null  float64       
 3   status             52516 non-null  object        
 4   country_code       47014 non-null  object        
 5   state_code         45753 non-null  object        
 6   region             46157 non-null  object        
 7   city               46157 non-null  object        
 8   funding_rounds     52516 non-null  int64         
 9   founded_at         52516 non-null  datetime64[ns]
 10  first_funding_at   52516 non-null  datetime64[ns]
 11  last_funding_at    52516 non-null  datetime64[ns]
 12  closed_at          4917 non-null   datetime64[ns]
dtypes: datetime64[ns](4), float64(1), int64(1), object(7)
memory 

In [17]:
def count_lifetime(el):
    if el['status'] == 'operating':
        return (pd.to_datetime('2018-01-01') - el["founded_at"]).dt.days
    else:
        return (el['closed_at'] - el["founded_at"]).dt.days
data['lifetime_2'] = data.apply(count_lifetime, axis=1)

AttributeError: 'Timedelta' object has no attribute 'dt'

In [12]:
# разница, выраженная в днях
data["lifetime"] = \
(pd.to_datetime('2018-01-01') - data["founded_at"]).dt.days
data

,name,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at,closed_at,lifetime
0,Lunchgate,Online Reservations|Restaurants,828626.00,operating,CHE,25,Zurich,Zürich,2,2009-10-17,2011-05-01,2014-12-01,NaT,2998
1,EarLens,Manufacturing|Medical|Medical Devices,42935019.00,operating,USA,CA,SF Bay Area,Redwood City,4,2005-01-01,2010-05-04,2014-02-25,NaT,4748
2,Reviva Pharmaceuticals,Biotechnology,35456381.00,operating,USA,CA,SF Bay Area,San Jose,3,2006-01-01,2012-08-20,2014-07-02,NaT,4383
3,Sancilio and Company,Health Care,22250000.00,operating,NaN,NaN,NaN,NaN,3,2004-01-01,2011-09-01,2014-07-18,NaT,5114
4,WireTough Cylinders,Manufacturing,NaN,operating,USA,VA,VA - Other,Bristol,1,2010-05-12,2012-02-01,2012-02-01,NaT,2791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52511,Videostream,Entertainment,NaN,operating,CAN,ON,Toronto,Kitchener,1,2012-01-01,2014-03-01,2014-03-01,NaT,2192
52512,Hello Curry,Hospitality,500000.00,operating,IND,2,Hyderabad,Hyderabad,1,2013-08-25,2014-03-07,2014-03-07,NaT,1590
52513,Taskforce,Email|Messaging|Productivity Software,50000.00,operating,USA,CA,SF Bay Area,San Francisco,3,2010-07-01,2009-06-14,2011-01-01,NaT,2741
52514,NetScaler,Security,13000000.00,operating,USA,CA,SF Bay Area,San Jose,6,1997-12-01,1998-11-30,2004-03-01,NaT,7336


In [13]:
data['status'].unique()

array(['operating', 'closed'], dtype=object)

In [14]:
#crystalsol = startups_test[startups_test['name'] == 'Crystalsol']

In [15]:
data.isna().sum()

name                     1
category_list         2465
funding_total_usd    10069
status                   0
country_code          5502
state_code            6763
region                6359
city                  6359
funding_rounds           0
founded_at               0
first_funding_at         0
last_funding_at          0
closed_at            47599
lifetime                 0
dtype: int64

In [16]:
len(data[data.duplicated(subset=[\
    'name'])])

0